In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import matplotlib.pyplot as plt
import tensorflow as tf
# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv("../input/sign_mnist_train.csv")
test=pd.read_csv("../input/sign_mnist_test.csv")

In [ ]:
train.head()             #datayı incele

In [ ]:
train.describe()                     #datayı incele

In [ ]:
train.info()                     #datayı incele

In [ ]:
train = train[train["label"]==1].append(train[train["label"]==2]).append(train[train["label"]==3]).append(train[train["label"]==4]).append(train[train["label"]==5])
test = test[test["label"]==1].append(test[test["label"]==2]).append(test[test["label"]==3]).append(test[test["label"]==4]).append(test[test["label"]==5])                                                                                                                                        
x_train = train.drop(["label"], axis=1).values.T
y_train = train[["label"]].values.reshape(-1,1).T
x_test =  test.drop(["label"], axis=1).values.T    # LABELLARI AYIR ARRAYE DÖNÜŞTÜR 
y_test =  test[["label"]].values.reshape(-1,1).T
train.head()

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y
y_train = convert_to_one_hot(y_train, 6)
y_test = convert_to_one_hot(y_test, 6)
x_train = x_train/255                           # NORMALIZE DATA
x_test = x_test/255   
           

In [ ]:
print("Shape of x_train",x_train.shape)
print("Shape of y_train",y_train.shape)
print("Shape of x_test",x_test.shape)
print("Shape of y_test",y_test.shape)              #Shape of DATA

In [ ]:
# GRADED FUNCTION: initialize_parameters_he
layer_dims= [x_train.shape[0],50,25,12,6]  
def initialize_params(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    
    parameters = {}
    L = len(layer_dims) - 1 # integer representing the number of layers
     
    for l in range(1, L + 1):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['w' + str(l)] = np.random.randn(layer_dims [l],layer_dims[l-1]) * np.sqrt(2 / layer_dims[l - 1])
        parameters['b' + str(l)] = np.zeros([layer_dims[l],1])
        ### END CODE HERE ###
        
    return parameters

In [ ]:
# Forward propagation
def relu(z):
    a = np.maximum(0,z)
    return a
def softmax(z):
    t = np.exp(z)
    summ=np.sum(t,axis=0)
    t=t/summ
    return t

def forward_Z(a_prev,w,b):         # Z DEĞERLERİNİ HESAPLAMA
    z = np.dot(w,a_prev) + b
    return z

def forward_activate(x, parameters):      # aktivasyon
    z_a_deposu = {}
    z_a_deposu["A0"] = x
    for l in range(1,int(len(parameters)/2)):
        z_a_deposu["z" + str(l)] = forward_Z(z_a_deposu["A" + str(l-1)],parameters["w" + str(l)] , parameters["b" + str(l)])
        z_a_deposu["A" + str(l)] = np.tanh(z_a_deposu["z" + str(l)]) 
    L = int(len(parameters)/2)
    z_a_deposu["z"+str(L)] = forward_Z(z_a_deposu["A" + str(L-1)],parameters["w" + str(L)] , parameters["b" + str(L)])
    z_a_deposu["A"+str(L)  ] = softmax(z_a_deposu["z" + str(L)])
    AL = z_a_deposu["A"+str(L)]
    return AL , z_a_deposu

def cost(AL,y):
    cost = -np.sum(y*np.log(AL))/y.shape[1]
    return cost

def forward_prop(x, y, parameters):
    AL, z_a_deposu = forward_activate(x,parameters)
    Cost = cost(AL,y)
    return AL, Cost , z_a_deposu        

In [ ]:
# BACKWARD PROPAGATIN

def relu_derrivative(a):
    da= a>0
    return da
def tanh_derrivative(A):
    dA = 1-A**2
    return dA
def softmax_derrivative(A):
    dA = A*(1-A)
    return dA
def backward(x, y, parameters):
    AL, cost , z_a_deposu = forward_prop(x, y, parameters)
    m = y.shape[1]            # NUMBER OF EXAMPLES
    türev_deposu = {}
    türev_deposu["dA" + str(int(len(parameters)/2))] = -np.divide(-y,AL)     #dJ/dAL 
    türev_deposu["dz" + str(int(len(parameters)/2))] = AL - y
    for l in reversed(range(1,int(len(parameters)/2) + 1)):
        türev_deposu["dw" + str(l)] = np.dot(türev_deposu["dz" + str(l)],z_a_deposu["A" + str(l-1)].T)/m
        türev_deposu["db" + str(l)] = np.sum(türev_deposu["dz" + str(l)], axis=1, keepdims=True)/m
        türev_deposu["dA" + str(l-1)] = np.dot(parameters["w" + str(l)].T , türev_deposu["dz" + str(l)])/m
        türev_deposu["dz" + str(l-1)] = türev_deposu["dA" + str(l-1)] * tanh_derrivative(z_a_deposu["A" + str(l-1)])
    return türev_deposu

In [ ]:
def update(X, parameters, z_a_deposu, türev_deposu, learning_rate):
    for l in range(1,len(layer_dims)):
        parameters["w" + str(l)] = parameters["w" + str(l)] - learning_rate * türev_deposu["dw" + str(l)]
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate * türev_deposu["db" + str(l)]
    return parameters

In [ ]:
layer_dims= [x_train.shape[0],50,25,12,6]     
def l_layer_model(x, y,x_test,y_test, layer_dims, learning_rate, num_of_iterations):
    parameters = initialize_params(layer_dims)
    AL, cost , z_a_deposu = forward_prop(x, y, parameters)
    türev_deposu =  backward(x, y, parameters)
    Cost1 = []
    Cost2 = []
    index = []
    for i in range(num_of_iterations):
        AL, cost , z_a_deposu = forward_prop(x, y, parameters)
        türev_deposu =  backward(x, y, parameters)
        parameters = update(x, parameters, z_a_deposu, türev_deposu, learning_rate)
        Cost1.append(cost)
        if i%250 ==0:
            Cost2.append(cost)
            index.append(i)
            print("cost after iter",i , "is" , cost)
    plt.plot(index,Cost2)
    plt.plot(index,Cost2)

    predicted_train = AL
    predicted_test = forward_prop(x_test, y_test, parameters)[0]
    train_accuracy = np.sum(np.equal(np.argmax(AL,axis=0), np.argmax(y_train,axis=0)))/y_train.shape[1]
    test_accuracy = np.sum(np.equal(np.argmax(predicted_test,axis=0), np.argmax(y_test,axis=0)))/y_test.shape[1]

    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    return AL

In [ ]:
l_layer_model(x_train, y_train,x_test,y_test, layer_dims, learning_rate=1, num_of_iterations=2500)

* > ***AS SEEN ACCURACY IS NOT ENOUGH HIGH.
* > LETS USE ADAM OPTIMIZER*****

In [ ]:
def initialize_adam(parameters) :
  
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s.
    for l in range(L):
        v["dW" + str(l + 1)] = np.zeros_like(parameters["w" + str(l + 1)])
        v["db" + str(l + 1)] = np.zeros_like(parameters["b" + str(l + 1)])

        s["dW" + str(l+1)] = np.zeros_like(parameters["w" + str(l + 1)])
        s["db" + str(l+1)] = np.zeros_like(parameters["b" + str(l + 1)])
    
    return v, s

In [ ]:
def update_parameters_with_adam(parameters, grads, v, s, t=2, learning_rate=0.01,
                                beta1=0.9, beta2=0.999, epsilon=1e-8):

    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    for l in range(L):
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".

        v["dW" + str(l + 1)] = beta1 * v["dW" + str(l + 1)] + (1 - beta1) * grads['dw' + str(l + 1)]
        v["db" + str(l + 1)] = beta1 * v["db" + str(l + 1)] + (1 - beta1) * grads['db' + str(l + 1)]


        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".

        v_corrected["dW" + str(l + 1)] = v["dW" + str(l + 1)] / (1 - np.power(beta1, t))
        v_corrected["db" + str(l + 1)] = v["db" + str(l + 1)] / (1 - np.power(beta1, t))


        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".

        s["dW" + str(l + 1)] = beta2 * s["dW" + str(l + 1)] + (1 - beta2) * np.power(grads['dw' + str(l + 1)], 2)
        s["db" + str(l + 1)] = beta2 * s["db" + str(l + 1)] + (1 - beta2) * np.power(grads['db' + str(l + 1)], 2)

        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".

        s_corrected["dW" + str(l + 1)] = s["dW" + str(l + 1)] / (1 - np.power(beta2, t))
        s_corrected["db" + str(l + 1)] = s["db" + str(l + 1)] / (1 - np.power(beta2, t))

        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
        parameters["w" + str(l + 1)] = parameters["w" + str(l + 1)] - learning_rate * v_corrected["dW" + str(l + 1)] / np.sqrt(s_corrected["dW" + str(l + 1)] + epsilon)
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * v_corrected["db" + str(l + 1)] / np.sqrt(s_corrected["db" + str(l + 1)] + epsilon)


    return parameters, v, s

In [ ]:
layer_dims= [x_train.shape[0],50,25,12,6]     
def l_layer_model(x, y,x_test,y_test, layer_dims, learning_rate, num_of_iterations):
    parameters = initialize_params(layer_dims)
    v , s = initialize_adam(parameters)
    AL, cost , z_a_deposu = forward_prop(x, y, parameters)
    türev_deposu =  backward(x, y, parameters)
    Cost1 = []
    Cost2 = []
    index = []
    for i in range(num_of_iterations):
        AL, cost , z_a_deposu = forward_prop(x, y, parameters)
        türev_deposu =  backward(x, y, parameters)
        parameters , v , s  =update_parameters_with_adam(parameters,türev_deposu,v,s)
        Cost1.append(cost)
        if i%250 ==0:
            print("cost after iter",i , "is" , cost)
        Cost2.append(cost)
        index.append(i)
            
    plt.plot(index,Cost2)

    predicted_train = AL
    predicted_test = forward_prop(x_test, y_test, parameters)[0]
    train_accuracy = np.sum(np.equal(np.argmax(AL,axis=0), np.argmax(y_train,axis=0)))/y_train.shape[1]
    test_accuracy = np.sum(np.equal(np.argmax(predicted_test,axis=0), np.argmax(y_test,axis=0)))/y_test.shape[1]
 
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    return AL

In [ ]:
 l_layer_model(x_train, y_train,x_test,y_test, layer_dims, learning_rate=1, num_of_iterations=3000)